In [11]:
%load_ext autoreload
%autoreload 2
from pymongo import MongoClient
import sys
from pathlib import Path
from tqdm import tqdm
import json
import random
import bson
from faker import Faker

sys.path.append(str(Path("..").resolve()))
from src import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Final touches: recover the lost Consistency due to seed
ℹ️ This notebook require the execution of [`1.2-Seeding_Database`](1.2-Seeding_Database.ipynb). Or you can just load the snapshop as in the following cell.

## Load post-preprocessing data
The following cell reset the `omero_museum` database loading the `3_seeded` snapshot.

In [12]:
%%capture
MongoClient("mongodb://localhost:27017/").drop_database("omero_museum")
!mongorestore --host localhost:27017 --drop --db omero_museum  "../backup/3_seeded/omero_museum"

In [13]:
faker=Faker()
connector = MongoDBConnector("omero_museum")
db = connector.db

The collections of the omero_museum db are:
----------------------------------------
[activities]: _id capacity duration enrolled room start_date ticketIds workshop_title
[artworks]: _id authorIds comments_star_1 comments_star_2 comments_star_3 comments_star_4 comments_star_5 date description donation_state donator_id is_original location_name materials period seller_id size tecniques trade type
[authors]: _id birth_date gender home_town name surname
[departments]: _id floor free_spots room
[limited_events]: _id artist artwork_titles author_ids capacity description end_date room_name start_date theme ticket_ids type
[messages]: _id content customer_id delivery_status length sent_date type uri
[roles]: _id activity_ids birth_date chat_ids curriculum date_start department desk email english_cert gender hometown name password phone_number salary shifts surname type username
[rooms]: _id floor
[suppliers]: _id email iban is_museum is_state name phone_num survey_ids
[visitors]: _id birth_da

## Artworks' comments' rating

In [14]:
for star in [1, 2, 3, 4, 5]:
    db.artworks.update_many({}, {"$set": {f"comments_star_{star}.$[].rating": star}})

## Visitors' `is_customer`

In [15]:
customer_fields = [
    "surname",
    "name",
    "donations_ids",
    "sales_ids",
    "phone_num",
    "email",
    "trade_ids",
    "chat",
    "comments",
]
db.visitors.update_many(
    {"is_customer": False},
    [
        {"$unset": customer_fields},
        {
            "$set": {
                "tickets": {"$slice": ["$tickets", 1]},
                "surveys": {"$slice": ["$surveys", 1]},
            }
        },
    ],
)

UpdateResult({'n': 51, 'nModified': 51, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

## Messages' `type`

In [16]:
for msg in db.messages.find({}):
    if msg["type"] == "audio":
        db.messages.update_one(
            {"_id": msg["_id"]},
            [
                {"$unset": ["uri", "size"]},
                {"$set": {"content": faker.hexify("^" * 100)}},
            ],
        )
    if msg["type"] == "picture":
        db.messages.update_one(
            {"_id": msg["_id"]},
            [
                {"$unset": ["content", "length"]},
                {"$set": {"uri": faker.image_url()}},
            ],
        )
    if msg["type"] == "text":
        db.messages.update_one(
            {"_id": msg["_id"]},
            [{"$unset": ["uri", "size"]}],
        )

## Suppliers' `is_museum`

In [19]:
for _ in [True, False]:
    db.suppliers.update_many(
        {"survey_ids": {"$exists": _}},
        [{"$set": {"is_museum": _}}],
    )

---
## Dump Final Database

In [20]:
%%capture
!mongodump --host localhost:27017 --db omero_museum --out "../backup/4_consistent"